In [4]:

import random

import numpy as np
import textgrad as tg

In [2]:
def set_seed(seed) :
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
from textgrad.engine_experimental.litellm import LiteLLMEngine

LiteLLMEngine("groq/openai/gpt-oss-120b" , cache=True).generate(content="hello, what's 3+4" , system_prompt="you are an assistant")

In [18]:
from openai import OpenAI
from textgrad.engine.local_model_openai_api import ChatExternalClient

# start a server with lm-studio and point it to the right address; here we use the default address.
client = OpenAI(base_url="https://api.groq.com/openai/v1" , api_key="")

engine = ChatExternalClient(client=client , model_string='openai/gpt-oss-120b')

In [19]:
tg.set_backward_engine(engine , override=True)

initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

solution = tg.Variable(initial_solution ,
                       requires_grad=True ,
                       role_description="solution to the math question")

loss_system_prompt = tg.Variable("""You will evaluate a solution to a math question.
Do not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise.""" ,
                                 requires_grad=False ,
                                 role_description="system prompt")

loss_fn = tg.TextLoss(loss_system_prompt)
optimizer = tg.TGD([ solution ])

In [20]:
loss = loss_fn(solution)
print(loss.value)

- The discriminant should be \(b^{2}-4ac = (-7)^{2}-4\cdot3\cdot2 = 49-24 = 25\), not \(49+24 = 73\).  
- The denominator \(2a = 6\) must be kept in the final expressions; the solutions are \(\displaystyle x=\frac{7\pm5}{6}\), i.e. \(x=2\) and \(x=\frac13\).  
- As written, the answers \((7\pm\sqrt{73})\) omit the division by 6 and use the wrong discriminant.


In [10]:
# print textgrad.engine info
print(tg.engine.info( ))

Engine model: mlabonne/NeuralBeagle14-7B-GGUF
Engine instance: <textgrad.engine.local_model_openai_api.ChatExternalClient object at 0x110a423c0>


In [12]:
import os

# env variable
os.environ[ 'GROQ_API_KEY' ] = ""

In [15]:
from litellm import completion

response = completion(
        model="groq/openai/gpt-oss-120b" ,
        messages=[
                { "role" : "user" , "content" : "hello from litellm" }
        ] ,
)
print(response)

ModelResponse(id='chatcmpl-04017b6a-6a30-4d40-8141-8d3aa9151b44', created=1767727120, model='openai/gpt-oss-120b', object='chat.completion', system_fingerprint='fp_fe269835c7', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hello! 👋 How can I assist you today?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None, reasoning='We need to respond. The user says "hello from litellm". Probably a greeting. Should respond friendly.'))], usage=Usage(completion_tokens=44, prompt_tokens=76, total_tokens=120, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=24, rejected_prediction_tokens=None, text_tokens=None, image_tokens=None), prompt_tokens_details=None, queue_time=0.044147329, prompt_time=0.003207882, completion_time=0.092844607, total_time=0.096052489), usage_breakdown=None, x_groq={'id': 'req_01keabygb4f05bfcyf935qxqaf', 'seed': 565054761}, service_tier='a

In [17]:
from textgrad.engine_experimental.litellm import LiteLLMEngine

LiteLLMEngine("groq/openai/gpt-oss-120b" , cache=True).generate(content="hello, what's 3+4" , system_prompt="you are an assistant")

'Hello!\u202fThe sum of\u202f3\u202fand\u202f4\u202fis\u202f7.'